# Prediction of the performance of students in a particular student based on the subject code(taking last 2 digits into considration) ,Output = GRADE .....Using ML libraries

### importing required modules

In [1]:
from pyspark.sql import SparkSession
import sys
import os
import matplotlib.pyplot as plt
import pandas as pd
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import OneHotEncoder

### creating Spark session

In [2]:
spark = SparkSession.builder.appName('Practise').getOrCreate()

22/04/15 18:45:46 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 192.168.0.195 instead (on interface wlp0s20f3)
22/04/15 18:45:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/15 18:45:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

### reading csv file

In [4]:
df = spark.read.csv('Results_Data.csv',header=True,inferSchema=True)
df.show()
df.count()

+----------+--------+---+---+---+------+-----+-----+--------+
|       USN|   SCODE|CIE|SEE|TOT|RESULT|GRADE| DATE|ExamType|
+----------+--------+---+---+---+------+-----+-----+--------+
|4JN18CS001| 18MAT11| 32| 41| 73|     P|    8|12019| Regular|
|4JN18CS001| 18PHY12| 29| 47| 76|     P|    8|12019| Regular|
|4JN18CS001| 18ELE13| 25| 43| 68|     P|    7|12019| Regular|
|4JN18CS001| 18CIV14| 31| 26| 57|     P|    6|12019| Regular|
|4JN18CS001|18EGDL15| 31| 56| 87|     P|    9|12019| Regular|
|4JN18CS001|18PHYL16| 32| 40| 72|     P|    8|12019| Regular|
|4JN18CS001|18ELEL17| 34| 52| 86|     P|    9|12019| Regular|
|4JN18CS001| 18EGH18| 27| 32| 59|     P|    6|12019| Regular|
|4JN18CS002| 18MAT11| 33| 51| 84|     P|    9|12019| Regular|
|4JN18CS002| 18PHY12| 36| 45| 81|     P|    9|12019| Regular|
|4JN18CS002| 18ELE13| 37| 31| 68|     P|    7|12019| Regular|
|4JN18CS002| 18CIV14| 34| 29| 63|     P|    7|12019| Regular|
|4JN18CS002|18EGDL15| 34| 55| 89|     P|    9|12019| Regular|
|4JN18CS

15000

In [5]:
import string
encoded1= {}
count =0
for i in string.ascii_uppercase:
    if i not in encoded1:
        encoded1[i]= count
    count = count +1
print (encoded1)

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25}


In [6]:
data_itr = df.rdd.toLocalIterator()
encoded = []

def replace_all1(text, encoded1):
    for i, j in encoded1.items():
        text = text.replace(i, str(j))
    return text

print(encoded1)

for i in data_itr:
    a =i['USN'] 
    b = i['SCODE']
    c = i['DATE']
    encoded.append({'USN1':a,'encodedUSN':int(replace_all1(a,encoded1)),'SCODE1':b,'DATE1':c})
        
print(encoded)






{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25}
[{'USN1': '4JN18CS001', 'encodedUSN': 491318218001, 'SCODE1': '18MAT11', 'DATE1': '12019'}, {'USN1': '4JN18CS001', 'encodedUSN': 491318218001, 'SCODE1': '18PHY12', 'DATE1': '12019'}, {'USN1': '4JN18CS001', 'encodedUSN': 491318218001, 'SCODE1': '18ELE13', 'DATE1': '12019'}, {'USN1': '4JN18CS001', 'encodedUSN': 491318218001, 'SCODE1': '18CIV14', 'DATE1': '12019'}, {'USN1': '4JN18CS001', 'encodedUSN': 491318218001, 'SCODE1': '18EGDL15', 'DATE1': '12019'}, {'USN1': '4JN18CS001', 'encodedUSN': 491318218001, 'SCODE1': '18PHYL16', 'DATE1': '12019'}, {'USN1': '4JN18CS001', 'encodedUSN': 491318218001, 'SCODE1': '18ELEL17', 'DATE1': '12019'}, {'USN1': '4JN18CS001', 'encodedUSN': 491318218001, 'SCODE1': '18EGH18', 'DATE1': '12019'}, {'USN1': '4JN18CS002', 'encodedUSN': 491318

### preprosessnig dataframe

In [7]:
dataframe = spark.createDataFrame(encoded)
dataframe.show()
dataframe.count()

+-----+--------+----------+------------+
|DATE1|  SCODE1|      USN1|  encodedUSN|
+-----+--------+----------+------------+
|12019| 18MAT11|4JN18CS001|491318218001|
|12019| 18PHY12|4JN18CS001|491318218001|
|12019| 18ELE13|4JN18CS001|491318218001|
|12019| 18CIV14|4JN18CS001|491318218001|
|12019|18EGDL15|4JN18CS001|491318218001|
|12019|18PHYL16|4JN18CS001|491318218001|
|12019|18ELEL17|4JN18CS001|491318218001|
|12019| 18EGH18|4JN18CS001|491318218001|
|12019| 18MAT11|4JN18CS002|491318218002|
|12019| 18PHY12|4JN18CS002|491318218002|
|12019| 18ELE13|4JN18CS002|491318218002|
|12019| 18CIV14|4JN18CS002|491318218002|
|12019|18EGDL15|4JN18CS002|491318218002|
|12019|18PHYL16|4JN18CS002|491318218002|
|12019|18ELEL17|4JN18CS002|491318218002|
|12019| 18EGH18|4JN18CS002|491318218002|
|12019| 18MAT11|4JN18CS003|491318218003|
|12019| 18PHY12|4JN18CS003|491318218003|
|12019| 18ELE13|4JN18CS003|491318218003|
|12019| 18CIV14|4JN18CS003|491318218003|
+-----+--------+----------+------------+
only showing top

15000

In [8]:
new = df.join(dataframe,(df.USN ==  dataframe.USN1) & (df.SCODE ==dataframe.SCODE1) & (df.DATE == dataframe.DATE1))
new.show()
new.drop('USN1').show()
new.count()

+----------+--------+---+---+---+------+-----+-----+--------+-----+--------+----------+------------+
|       USN|   SCODE|CIE|SEE|TOT|RESULT|GRADE| DATE|ExamType|DATE1|  SCODE1|      USN1|  encodedUSN|
+----------+--------+---+---+---+------+-----+-----+--------+-----+--------+----------+------------+
|4JN18CS001| 18MAT11| 32| 41| 73|     P|    8|12019| Regular|12019| 18MAT11|4JN18CS001|491318218001|
|4JN18CS001| 18PHY12| 29| 47| 76|     P|    8|12019| Regular|12019| 18PHY12|4JN18CS001|491318218001|
|4JN18CS001| 18ELE13| 25| 43| 68|     P|    7|12019| Regular|12019| 18ELE13|4JN18CS001|491318218001|
|4JN18CS001| 18CIV14| 31| 26| 57|     P|    6|12019| Regular|12019| 18CIV14|4JN18CS001|491318218001|
|4JN18CS001|18EGDL15| 31| 56| 87|     P|    9|12019| Regular|12019|18EGDL15|4JN18CS001|491318218001|
|4JN18CS001|18PHYL16| 32| 40| 72|     P|    8|12019| Regular|12019|18PHYL16|4JN18CS001|491318218001|
|4JN18CS001|18ELEL17| 34| 52| 86|     P|    9|12019| Regular|12019|18ELEL17|4JN18CS001|4913

15000

In [9]:
new.createOrReplaceTempView("df2_view")

dffinal = spark.sql(
    """
    SELECT encodedUSN,GRADE,
    (CASE
    WHEN SCODE LIKE '%1' THEN 1
    WHEN SCODE LIKE '%2' THEN 2
    WHEN SCODE LIKE '%3' THEN 3
    WHEN SCODE LIKE '%4' THEN 4
    WHEN SCODE LIKE '%5' THEN 5
    WHEN SCODE LIKE '%6' THEN 6
    WHEN SCODE LIKE '%7' THEN 7
    WHEN SCODE LIKE '%8' THEN 8
    WHEN SCODE LIKE '%9' THEN 9
    END) AS SCODEFINAL,
    (CASE
    WHEN DATE LIKE '%19' THEN 19
    WHEN DATE LIKE '%20' THEN 20
    END) AS NEWDATE
    
    FROM df2_view
    """
)

dffinal.show()

+------------+-----+----------+-------+
|  encodedUSN|GRADE|SCODEFINAL|NEWDATE|
+------------+-----+----------+-------+
|491318218001|    8|         1|     19|
|491318218001|    8|         2|     19|
|491318218001|    7|         3|     19|
|491318218001|    6|         4|     19|
|491318218001|    9|         5|     19|
|491318218001|    8|         6|     19|
|491318218001|    9|         7|     19|
|491318218001|    6|         8|     19|
|491318218002|    9|         1|     19|
|491318218002|    9|         2|     19|
|491318218002|    7|         3|     19|
|491318218002|    7|         4|     19|
|491318218002|    9|         5|     19|
|491318218002|    8|         6|     19|
|491318218002|    9|         7|     19|
|491318218002|    8|         8|     19|
|491318218003|    9|         1|     19|
|491318218003|    8|         2|     19|
|491318218003|    9|         3|     19|
|491318218003|    8|         4|     19|
+------------+-----+----------+-------+
only showing top 20 rows



In [10]:
# qualification_indexer = StringIndexer(inputCol="USN", outputCol="EncodedUSN")
# #Fits a model to the input dataset with optional parameters.
# df1 = qualification_indexer.fit(dffinal).transform(dffinal)
# df1.show()


In [11]:
#dffinal.printSchema

In [12]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=['encodedUSN','NEWDATE','SCODEFINAL'],outputCol="NEW")
output=featureassembler.transform(dffinal)
output.show()

+------------+-----+----------+-------+--------------------+
|  encodedUSN|GRADE|SCODEFINAL|NEWDATE|                 NEW|
+------------+-----+----------+-------+--------------------+
|491318218001|    8|         1|     19|[4.91318218001E11...|
|491318218001|    8|         2|     19|[4.91318218001E11...|
|491318218001|    7|         3|     19|[4.91318218001E11...|
|491318218001|    6|         4|     19|[4.91318218001E11...|
|491318218001|    9|         5|     19|[4.91318218001E11...|
|491318218001|    8|         6|     19|[4.91318218001E11...|
|491318218001|    9|         7|     19|[4.91318218001E11...|
|491318218001|    6|         8|     19|[4.91318218001E11...|
|491318218002|    9|         1|     19|[4.91318218002E11...|
|491318218002|    9|         2|     19|[4.91318218002E11...|
|491318218002|    7|         3|     19|[4.91318218002E11...|
|491318218002|    7|         4|     19|[4.91318218002E11...|
|491318218002|    9|         5|     19|[4.91318218002E11...|
|491318218002|    8|    

In [13]:
df2 = output.select('encodedUSN','NEW','GRADE')
limited = df2.limit(2716)

### ML training starts 

In [14]:
train, test = limited.randomSplit([0.9, 0.1])


In [15]:
rf = RandomForestClassifier(featuresCol = 'NEW', labelCol = 'GRADE',maxBins=720)
rfModel = rf.fit(train)

22/04/15 18:45:57 ERROR Executor: Exception in task 6.0 in stage 24.0 (TID 41)
org.apache.spark.SparkException: Failed to execute user defined function (VectorAssembler$$Lambda$3663/0x0000000841446840: (struct<encodedUSN_double_VectorAssembler_e2799486ba2e:double,NEWDATE_double_VectorAssembler_e2799486ba2e:double,SCODEFINAL_double_VectorAssembler_e2799486ba2e:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:136)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage2.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.sc

Py4JJavaError: An error occurred while calling o96.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 6 in stage 24.0 failed 1 times, most recent failure: Lost task 6.0 in stage 24.0 (TID 41) (192.168.0.195 executor driver): org.apache.spark.SparkException: Failed to execute user defined function (VectorAssembler$$Lambda$3663/0x0000000841446840: (struct<encodedUSN_double_VectorAssembler_e2799486ba2e:double,NEWDATE_double_VectorAssembler_e2799486ba2e:double,SCODEFINAL_double_VectorAssembler_e2799486ba2e:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:136)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage2.processNext(Unknown Source)
	at org.apache.

In [ ]:
predictions = rfModel.transform(test)
finalres = predictions.select(['encodedUSN','GRADE','prediction']).show(25)

In [ ]:
predictions.select("GRADE", "prediction").show(10)

In [ ]:
dftest = spark.read.csv('new.csv',header=True,inferSchema=True)
dftest.show()
dftest = dftest.limit(15000)

In [ ]:
qualification_indexer = StringIndexer(inputCol="USN", outputCol="EncodedUSN")
#Fits a model to the input dataset with optional parameters.
dftest1 = qualification_indexer.fit(dftest).transform(dftest)
dftest1.show()

In [ ]:
dftest1.createOrReplaceTempView("df3_view")

dffinal2 = spark.sql(
    """
    SELECT EncodedUSN,GRADE,
    (CASE
    WHEN SCODE LIKE '%1' THEN 1
    WHEN SCODE LIKE '%2' THEN 2
    WHEN SCODE LIKE '%3' THEN 3
    WHEN SCODE LIKE '%4' THEN 4
    WHEN SCODE LIKE '%5' THEN 5
    WHEN SCODE LIKE '%6' THEN 6
    WHEN SCODE LIKE '%7' THEN 7
    WHEN SCODE LIKE '%8' THEN 8
    WHEN SCODE LIKE '%9' THEN 9
    END) AS SCODEFINAL,
    (CASE
    WHEN 2021 LIKE '%21' THEN 21
    END) AS NEWDATE
    
    FROM df3_view
    """
)

dffinal2.show()

In [ ]:
featureassembler=VectorAssembler(inputCols=['EncodedUSN','NEWDATE','SCODEFINAL'],outputCol="NEW")
output1=featureassembler.transform(dffinal2)
output1.show()

In [ ]:
testfinal = output1.select('NEW','GRADE')

In [ ]:
df1.select('EncodedUSN').show(50)

In [ ]:
StringIndexer(inputCols=["USN"], outputCols=["newUSN1"])
encoder = OneHotEncoder(
    inputCols=["newUSN1"],  
    outputCols=["newUSN"]
)

In [ ]:
indexer = StringIndexer(inputCol="USN", outputCol="newUSN").fit(dffinal)
indexed_df = indexer.transform(dffinal)
indexed_df.show()

### Prediction

In [ ]:
predictions = rfModel.transform(testfinal)
predictions.select(['NEW','GRADE','prediction']).show(25)

In [ ]:
mPath = "Model1"
rfModel.write().overwrite().save(mPath)